In [ ]:
import sys,os

from larcv import larcv
import ROOT

import matplotlib
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from ROOT import geo2d
pygeo = geo2d.PyDraw()

%matplotlib inline

In [ ]:
iom=larcv.IOManager()
#...nu_e...
iom.add_in_file("/Users/vgenty/Desktop/intrinsic_nue/out_pyroi/larcv_fcn_out_0000_0099.root")
iom.initialize()
tf=ROOT.TFile("/Users/vgenty/Desktop/intrinsic_nue/out_pyroi/c_ana_0000_0099.root")
#...nu_nc...
recotree=tf.RecoTree

In [ ]:
#...nu_e...
#entry 14 -- 1 plane good
#entry 23 -- 2 plane good
#entry 38 -- plane 2 trunk is classified as track, plane 0, none of it is, dead wire region @ vertex plane 1
#entry 45 -- 2 plane good
#entry 51 -- 2 plane good
#...nu_nc...
recotree.GetEntry(6) 
print "Verticies: ", recotree.Vertex3D_v.size()
run    = recotree.run
subrun = recotree.subrun
event  = recotree.event
entry  = recotree.entry
print "RSEE: ",run,subrun,event,entry

In [ ]:
iom.read_entry(entry)
evimg=iom.get_data(larcv.kProductImage2D,"tpc")
img_v = [evimg.Image2DArray().at(i) for i in xrange(3)]

for vtxid in xrange(recotree.Vertex3D_v.size()):
    vertex=recotree.Vertex3D_v.at(vtxid)
    print "<==START======================>"
    print "<=====Vertex %d===============>"%vtxid
    print "<==============Type: %d=======>"%vertex.type
    pcluster_vv=recotree.ParticleCluster_vvv.at(vtxid)
    tcluster_vv=recotree.TrackClusterCompound_vvv.at(vtxid)
    meta_v = recotree.Meta_vv.at(vtxid)
    print vertex.x,vertex.y,vertex.z
    for plane in xrange(3):
        print "-----------Plane is %d------------" % plane
        meta=meta_v.at(plane)
        
        fig,ax=plt.subplots(figsize=(20,20))
        img2d=img_v[plane]
        img_crop=img2d.crop(meta)
        img=larcv.as_ndarray(img_crop)
        img[img>15]=255
        img=img[:,::-1]
        
        
        
        pcluster_v=pcluster_vv.at(plane)
        tcluster_v=tcluster_vv.at(plane)
        xmin_=9e9
        xmax_=0
        ymin_=9e9
        ymax_=0
        
        dqdx_pv=[]
        pca_pv=[]
        for parid,pcluster in enumerate(pcluster_v):
            ctor=[[pt.x,pt.y] for pt in pcluster._ctor]
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            if xmin_ > np.min(ctor[:,0]): xmin_ = np.min(ctor[:,0])
            if xmax_ < np.max(ctor[:,0]): xmax_ = np.max(ctor[:,0])
            if ymin_ > np.min(ctor[:,1]): ymin_ = np.min(ctor[:,1])
            if ymax_ < np.max(ctor[:,1]): ymax_ = np.max(ctor[:,1])
                
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=2)
            
            if pcluster.type==1:
                tcluster = tcluster_v[parid]
                print "On",parid,"got",tcluster.size(),"atomics"
                dqdx_v=[]
                pca_v=[]
                for atom in tcluster.get_ordered_atoms():
                    dqdx_v_=list(np.array(atom.dqdx()))
                    dqdx_v+=dqdx_v_
                    pca_v.append(atom.pca())
                dqdx_v=np.array(dqdx_v)
                dqdx_pv.append(dqdx_v)
                pca_pv.append(pca_v)
                    
        
        ax.imshow(img,cmap='Greys',interpolation='none')
        ax.set_aspect(1/3.0)
        for pca_a in pca_pv:
            for pca in pca_a:
                ax.plot([xmin_,xmax_],[pca.y(xmin_),pca.y(xmax_)],'-',color='Grey')
        
        plt.show()
        if len(dqdx_pv)>0:
            fig,ax=plt.subplots(figsize=(10,5))
            ctr=0
            for dqdx in dqdx_pv:
                x_=np.arange(ctr,ctr+dqdx.size,1)
                ax.plot(x_,dqdx,'-o')
                roll_=pd.rolling_mean(dqdx,5)
                ax.plot(np.arange(ctr,ctr+roll_.size,1),roll_,'-',color='red',lw=2)
                ctr+=dqdx.size
            ax.grid()
            plt.show()
        
    print "<=============================>"
    print "<=====Vertex %d ==============>"%vtxid
    print "<=====================END=====>"